## Projektstruktur

Ausgehend von einem beliebigen Wurzelordner liegt dem Projekt folgende Ordnerstruktur zugrunde:

- **master_cloud** [Sciebo-Ordner]
  - ... externer Code, Tutorials etc.
  - *corpora*
    - ... diverse rohe Corpora
    - hpc [backup der ETL und NLP Pipeline Resultate auf dem HPC]
    - *preprocessed* [corpora nach ETL Pipeline]
      - *nlp* [corpora nach NLP Pipeline]
        - spacy_model
          - vocab
    - scrapy
      - faz
      - focus
    + German_datasets.ods [s.u.]
    
- **topiclabeling**
  - data
    - feeds [Rohoutput der scrapy feeds]
    + IWNLP.Lemmatizer_20170501.json
  - logs
  - *src* [Git Projektordner, Python code und Notebooks]
    - scrapy [code]
  + ToDo.txt

    

In [2]:
from os import listdir, makedirs
from os.path import isfile, join

from constants import DATA_BASE, ETL_PATH, NLP_PATH, HASH
import pandas as pd
import ezodf

pd.set_option('display.precision',10)

## German Corpora Overview

Übersicht über deutschsprachige Corpora, die zur Verfügung stehen. Die ersten 6 fallen hierbei in die engere Wahl. Die gecrawlten News-Corpora von FAZ und Focus können auch als ein gemeinsamer Corpus betrachten werden, ebenso könnten auch weitere Newsseiten hinzugenommen werden. Das Parsing des Wikipedia-Corpus ist noch nicht abgeschlossen. Der dewac-Corpus ist aufgrund seiner Größe noch nicht vollständig durch die NLP-Pipeline gegangen.

In [8]:
def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()}).iloc[:,:-1].dropna(how='all')

def highlight(s):
    if s.Selected:
        col = '#eccc68' if s.name%2 else '#ecdf68'
        return ['background-color: %s' % col]*len(s)
    return ['']*len(s)

df = read_ods(join(DATA_BASE, 'German_datasets.ods'))
df.style.apply(highlight, axis=1)

,Category,Name,Date,Content,#documents,#sentences,#tokens,Size (unzipped + unprocessed),Remarks,Link,Selected
0,Online-Partizipation,Online-Partizipation,2011-2018,Online-Partizipation,25981,143384,1751790,16M,interessant,https://github.com/Liebeck/OnlineParticipationDatasets,True
1,Politics,German political speeches,1984-2018,includes the four highest ranked functions on federal state level,6037,221397,4136463,73M,interessant,https://hal.archives-ouvertes.fr/hal-01798703,True
2,Politics,Europarl,1996-2011,parallel corpus extracted from the European Parliament web site,12788,2692391,56609511,~3.5G,interessant,"http://opus.nlpl.eu/Europarl.php, http://www.statmt.org/europarl/",True
3,News,faz.de,2006-2018,"finanzen, feuilleton, gesellschaft, technik, politik, wirtschaft, wissen, reise, beruf",51385,1837668,29527792,201M,crawled,http://www.faz.de,True
4,News,focus.de,bis 2018,"politik, finanzen, wissen, gesundheit, [ongoing: kultur, panorama, digital, reisen, auto, immobilien, regional]",40746 [+x],None,None,82M [+x],crawled [ongoing],http://www.focus.de,True
5,Wikipedia,dewiki,bis 2018,German Wikipedia,None,None,None,19.7G,interessant,https://dumps.wikimedia.org/backup-index.html,True
6,www,dewac_preproc,bis 2018,constructed from the Web limiting the crawl to the .de domain and using medium-frequency words from the SudDeutsche Zeitung corpus and basic German vocabulary lists as seeds,1752094,None,None,10.2G,"interessant, aber wenige Metadaten",http://wacky.sslmit.unibo.it/doku.php?id=corpora#german,True
7,Div.,Wikisource,None,"Sammlung von Texten und Quellen, die entweder urheberrechtsfrei sind oder unter einer freien Lizenz stehen.",None,None,None,1.9G,"interessant, teilw. sprachlich veraltet, schwierig zu parsen (MediaWiki Markdown; enthält auch Meta Content)",https://de.wikisource.org/wiki/Hauptseite,False
8,Div.,DEREKO / IDS Mannheim,None,"belletristische, wissenschaftliche und populärwissenschaftliche Texte, eine große Zahl von Zeitungstexten sowie eine breite Palette weiterer Textarten",None,None,42B,None,nicht als Download verfügbar,http://www1.ids-mannheim.de/direktion/kl/projekte/korpora.html,False
9,Div.,DWDS-Kernkorpus,None,ein nach Textsorten und zeitlich über das gesamte Jahrhundert ausgewogenes Korpus,None,None,None,None,nicht als Download verfügbar,https://www.dwds.de/d/k-referenz/,False


In [4]:
def read(f):
     return pd.read_pickle(f)

## Scrapy spiders

Unter ```../../topiclabeling/src/scrapy``` sind derzeit zwei Spiders für die FAZ und den Focus lauffähig. Der Crawler lässt sich von dort aufrufen durch

```scrapy crawl [-a categories="politik|kultur|panorama|digital"] focus```

Die Categories beziehen sich auf die möglichen Rubriken der jeweiligen News-Seite. Ohne ihre Angabe werden als Default-Kategorien Politik und Wirtschaft gecrawlt.

## ETL Pipeline

Die rohen Corpora wurden durch [ETL_pipeline.ipynb](ETL_pipeline.ipynb) geladen, wenn nötig geparst und in ein einheitliches Datenschema gebracht, wobei unter 'Load' hier das Speichern als pandas DataFrame im pickle-Format zu verstehen ist. Der Output der Pipeline befindet sich im Ordner ```../../master_cloud/corpora/preprocessed```.

In [4]:
read(join(ETL_PATH, 'FAZ.pickle'))

,dataset,subset,doc_id,doc_subid,title,tags,date_time,text
hash,,,,,,,,
-3019970016641425166,FAZ,politik,http://www.faz.net/aktuell/politik/inland/heik...,0,„Balancierte Partnerschaft“: Maas plädiert für...,"(Heiko Maas, Bundesregierung, Handelsblatt, US...",2018-08-22 11:11:45+02:00,„Balancierte Partnerschaft“: Maas plädiert für...
8488528065330994748,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/wohnen/g...,0,Anwesen „Gravetye Manor“: Schmaler Grat der wi...,"(Tom Coward, William Robinson, Gravetye Manor ...",2018-08-02 10:34:07+02:00,Anwesen „Gravetye Manor“: Schmaler Grat der wi...
-3378811050728393830,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/unterneh...,0,Li Shufu aus China: Daimlers Großaktionär verd...,"(Daimler, ISIN_DE0007100000, Geely Internation...",2018-08-22 11:56:35+02:00,Li Shufu aus China: Daimlers Großaktionär verd...
-5730355306167585609,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/arm-und-...,0,Umfrage von EU-Statistikern: 13 Millionen Deut...,"(Sabine Zimmermann, Eurostat, Die Linke, Urlau...",2018-07-19 11:17:19+02:00,Umfrage von EU-Statistikern: 13 Millionen Deut...
925414413094503716,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/der-hand...,0,Überraschende Aussage: IW-Chef Hüther findet m...,"(Michael Hüther, Reuters, Institut der deutsch...",2018-08-08 08:13:22+02:00,Überraschende Aussage: IW-Chef Hüther findet m...
-3703467378027267948,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/kuenstli...,0,Handelskammer rechnet vor: Roboterautos erspar...,"(Georg Merziger, Reuters, BamS, DIHK, General ...",2018-07-15 10:58:41+02:00,Handelskammer rechnet vor: Roboterautos erspar...
8587015097104579414,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/diesel-a...,0,Autohersteller: Nissan räumt Fehler in Abgaste...,"(Nissan, Reuters, Renault, ISIN_FR0000131906, ...",2018-07-09 14:45:34+02:00,Autohersteller: Nissan räumt Fehler in Abgaste...
3594680563037495584,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/eurokris...,0,Asyl und Euro: Merkel spaltet die Europäische ...,"(Macron Wunsch, Angela Merkel, Bundeskanzler, ...",2018-06-21 08:33:07+02:00,Asyl und Euro: Merkel spaltet die Europäische ...
6522106565180681564,FAZ,wirtschaft,http://www.faz.net/aktuell/wirtschaft/diginomi...,0,Archivierung von Videospielen: Stirbt Pacman aus?,"(Nintendo, Love, Videospiel, Videospiele, Game...",2018-08-22 07:23:08+02:00,Archivierung von Videospielen: Stirbt Pacman a...


## NLP Pipeline

Nachdem die Corpora in ein einheitliches Format gebracht wurden durchlaufen sie die Spacy Pipeline für das 'de' language model. s. [NLP_pipeline.ipynb](NLP_pipeline.ipynb)

Die Tokenisierung ist nicht optimal und könnte durch Definition einiger Sonderfälle noch verbessert werden. Dies betrifft insbesondere den Forums-Corpus der Online-Partizipation. Hier macht es sich nachteilig bemerkbar, dass das Sprachmodell auf einem orthographisch korrekten Corpus trainiert wurde.

Die Lemmatisierung wird durch IWNLP durchgeführt. Um einige merkwürdige Ergebnisse abzufangen, habe ich hier einen Wrapper herum gebaut, der bestimmte Sonderfälle abfängt, bzw. bestimmte Wortarten nicht durch IWNLP parsen lässt. Zusätzlich überprüfe ich Suffixe von zusammengesetzten Nomen auf Lemmata. Wo IWNLP keine Ergebnisse findet, wird die Lemmatisierung des Spacy Modells verwendet, ansonsten das ursprüngliche Token. Für IWNLP verwende ich die XML-Datenbank von 2017. Der Versuch, das aktuelle Wiktionary neu zu parsen, schlug nach wenigen Wörtern mit einer OutOfMemory Exception fehl (Build sowohl unter Mono als auch Visual Studio). Die Lemmatisierung ist noch anfällig für Fehler, insbesondere wenn das POS-Tagging nicht korrekt war. Named Entities sind ein Problem, wenn sie von Spacy nicht als solche erkannt wurden.

Der Output der Pipeline befindet sich im Ordner ```../../master_cloud/corpora/preprocessed/nlp```. Das spacy vocabulary wird mit gespeichert.

In [5]:
read(join(NLP_PATH, 'FAZ_nlp.pickle'))

,hash,index,sent_idx,text,token,POS,ent_iob,noun_phrase
0,-3019970016641425166,0,0,„,„,PUNCT,O,0
1,-3019970016641425166,1,0,Balancierte,balancieren,ADJ,O,0
2,-3019970016641425166,2,0,Partnerschaft,Partnerschaft,NOUN,O,0
3,-3019970016641425166,3,0,“,“,PUNCT,O,0
4,-3019970016641425166,4,0,:,:,PUNCT,O,0
5,-3019970016641425166,5,1,Maas,maa,ADV,B,0
6,-3019970016641425166,6,1,plädiert,plädieren,VERB,O,0
7,-3019970016641425166,7,1,für,für,ADP,O,0
8,-3019970016641425166,8,1,neue,neu,ADJ,O,0
9,-3019970016641425166,9,1,Amerika-Strategie,Amerika-strategie,NOUN,O,0


## Phrase detection

Zu den Token des Corpus versuche ich Phrasen-Token zu generieren. Die Phrase-Detection von Gensim erwies sich dabei als eher problematisch. Die Ergebnisse sind zu willkürlich. Die Named Entities von Spacy liefern zu wenige Ergebnisse. Einigermaßen brauchbar (nach etwas Überarbeitung) sind Spacy's noun chunks. Interessant wäre hier ein Dictionary basierter Ansatz, z.B. auf Basis von Wikipedia-Titeln. Dies könnte aber rechenintensiv werden.

## Topic Modeling

Für das topic modeling werden nur die Tags NOUN, PROPN (Entities) und Phrasen verwendet. Zur Erstellung der Models wird Gensims LDAModel verwendet. Hier muss noch mit den Parametern experimentiert werden. Für das Topic Labeling wäre es wünschenswert möglichst exklusive Topics zu haben, d.h. mit hoher Wahrscheinlichkeitsmasse an den Rändern der Dirichlet-Verteilung, z.B. durch niedrige $\alpha$-Werte. Hier besteht auf jeden Fall noch Experimentierbedarf.

[Gensim_topic_modeling.ipynb](Gensim_topic_modeling.ipynb)

-----

### Appendix: Corpus metrics

In [5]:
path = NLP_PATH

files = sorted([f for f in listdir(path) if isfile(join(path, f))
                # and f[:3] == 'dew'
                # and f[:3] == 'Onl'
                # and f[:3] == 'Eur'
                # and f[:3] == 'FAZ'
                # and f[:3] == 'Pol'
               ])

dfs = [(name, read(join(path, name))) for name in files]

In [6]:
overview_list = [(name, df.groupby(HASH).ngroups, df.groupby([HASH, 'sent_idx']).ngroups, df.shape[0]) 
                 for name, df in dfs]
overview = pd.DataFrame(overview_list, columns=['dataset', '#documents', '#sentences', '#tokens'])
overview

,dataset,#documents,#sentences,#tokens
0,Europarl_nlp.pickle,12788,2692391,56609511
1,FAZ_nlp.pickle,51385,1837668,29527792
2,OnlineParticipation_nlp.pickle,25981,143384,1751790
3,PoliticalSpeeches_nlp.pickle,6037,221397,4136463


In [7]:
del dfs